In [1]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device = "cuda"
    print("CUDA device:", torch.cuda.get_device_name(0))
    print("Device count:", torch.cuda.device_count())
else:
    device = "cpu"
    print("Running on CPU only.")

Torch version: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla T4
Device count: 1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import shutil
project_root = 'drive/MyDrive/Agent/orchestrator-agent/'
model_path = project_root + 'orchestrator-model/TinyLlama-1.1B-Chat-v1.0/'

shutil.copy(project_root + 'preprocess_dataset.py', './')

'./preprocess_dataset.py'

In [21]:
shutil.copytree(project_root + 'datasets', './datasets')

'./dataset'

## Preprocess Raw Data

In [36]:
import preprocess_dataset as pre
from datasets import Dataset

#init
dataset_root = 'datasets/raw/en/'
dataset_save_root = 'datasets/formatted/en/'

In [39]:
pip install fsspec==2023.9.2 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requir

### Tool Memorization and Classification Dataset Preprocess

#### 1.  send_email tool dataset preprocessing

In [37]:
send_email_dataset_raw : Dataset = pre.load_jsonl_dataset(dataset_root + "send_email.jsonl")
send_email_dataset_formatted : dict = pre.format_dataset(send_email_dataset_raw)

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [26]:
# save
send_email_formatted_output_path = dataset_save_root + "send_email.jsonl"

saving_send_email_dataset_isSuccess : bool = pre.save_dataset(send_email_dataset_formatted, send_email_formatted_output_path)
if saving_send_email_dataset_isSuccess :
    print("Successfully saved dataset for sending email tool classification")

Error occurred: 'dict' object has no attribute 'to_json'


## Train Model

In [ ]:
#init
from transformers import AutoTokenizer

model_path = "./orchestrator-model/TinyLlama-1.1B-Chat-v1.0/" # or HuggingFace model ID
tokenizer = AutoTokenizer.from_pretrained(model_path)


### Tool Memorization and Classification Model Training

#### 1. send_email tool classification training

In [ ]:
# Tokenization
def tokenize_function(example):
    result = tokenizer(
        example["prompt"] + " " + example["response"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = send_email_dataset_formatted.map(tokenize_function, remove_columns=send_email_dataset_formatted.column_names)


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./orchestrator-model-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=True,
    fp16=True,
    gradient_accumulation_steps=4
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)


In [ ]:
print(tokenized_dataset[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


C:\Users\Fadhl\AppData\Local\Temp\ipykernel_9684\1889173795.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyboardInterrupt: 